In [2]:
# stack (into time series) and seperate (different bands) the tif images of NDVI EVI from MODIS
# 
from osgeo import gdal
import numpy as np

ds = gdal.Open('/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-02-18.tif')
ds


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f016c037db0> >

In [3]:
# loop through each band
for bi in range(ds.RasterCount):
    band = ds.GetRasterBand(bi + 1)
    # Read this band into a 2D NumPy array
    ar = band.ReadAsArray()
    print('Band %d has type %s'%(bi + 1, ar.dtype))
    raw = ar.tostring()

Band 1 has type int16
Band 2 has type int16
Band 3 has type int16


In [4]:
ar

array([[1014,  967,  967, ...,    0,    0,    0],
       [1006,  990,  967, ...,    0,    0,    0],
       [1006,  990,  990, ...,    0,    0,    0],
       ..., 
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int16)

In [42]:
outvrt = '/vsimem/stacked.vrt' #/vsimem is special in-memory virtual "directory"
outtif = '/tmp/stacked.tif'
a = '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-02-18.tif'
b = '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-03-05.tif'
c = '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-03-21.tif'
d = '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-02-18.tif'
tifs = [a, b, c, d] 
#or for all tifs in a dir
#import glob
#tifs = glob.glob('dir/*.tif')

outds = gdal.BuildVRT(outvrt, tifs, separate=True)
outds = gdal.Translate(outtif, outds)
outds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f015523ff90> >

In [41]:
outds.RasterCount
#outds.GetRasterBand

AttributeError: 'str' object has no attribute 'RasterCount'

In [16]:
for bi in range(outds.RasterCount):
    bandout = outds.GetRasterBand(bi + 1)
    # Read this band into a 2D NumPy array
    outar = bandout.ReadAsArray()
    print('Band %d has type %s'%(bi + 1, ar.dtype))
    raw = outar.tostring()

Band 1 has type int16
Band 2 has type int16
Band 3 has type int16
Band 4 has type int16


In [14]:
outar

array([[63, 63, 63, ...,  0,  0,  0],
       [63, 63, 63, ...,  0,  0,  0],
       [63, 63, 63, ...,  0,  0,  0],
       ..., 
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int16)

In [44]:
import rasterio
import glob
dir = '/home/599/rg6346/Downloads/MODIS_NDVI_EVI'
#tifs = glob.glob('dir/*.tif')
#file_list = tifs
file_list = [a, b, c, d]

# Read metadata of first file
with rasterio.open(file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(file_list))

# Read each layer and write it to stack
with rasterio.open('stack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(file_list):
        with rasterio.open(layer) as src1:
            dst.write_band(id + 1, src1.read(1))

In [47]:
meta

{'count': 4,
 'crs': CRS({'init': 'epsg:4326'}),
 'driver': 'GTiff',
 'dtype': 'int16',
 'height': 3147,
 'nodata': None,
 'transform': Affine(0.004162008945837755, 0.0, 138.5657638337764,
       0.0, -0.004162008945837755, -24.58498684306362),
 'width': 3346}

In [46]:
dst
#tifs
#file_list

<closed DatasetWriter name='stack.tif' mode='w'>